# Create Pixel Dataset
This notebook is the source for downloading Sentinel data to produce inputs to the spectral classifier.

## Inputs
The notebook operates by loading a set of sampling sites from a geojson. If the geojson contains `Point` features, a bounding rect is constructed. If the geojson contains `Polygon` or `MultiPolygon` features, only pixels within the polygon will be extracted.

The `download_patch` script attempts to mask clouds. However, cloudy pixels and patches can still come through.

Pixels that fall outside of a polygon are also masked using a numpy masked array. These vectors are not stored in the pixel vector list.

## Outputs

### Pixel Vectors:
The output is a list of vectors saved as a pickle. The vectors are not normalized. The dimensionality is `[num_pixel_vectors][bands]`
### Image Plot:
To log the data in a pixel vector dataset, a grid of input images is exported along with the datset.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import descarteslabs as dl
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

from scripts.viz_tools import normalize, band_descriptions, plot_image_grid
from scripts.dl_utils import download_patch, rect_from_point

In [ ]:
def save_pixel_vectors(data, name, label_class):
    file_name = f"{name}_{START_DATE}_{END_DATE}"
    with open(os.path.join(OUTPUT_DIR, f"{file_name}_pixel_vectors.pkl"),"wb") as f:
        pickle.dump(data, f)
    with open(os.path.join(OUTPUT_DIR, f"{file_name}_pixel_vector_labels.pkl"),"wb") as f:
        pickle.dump([label_class] * len(data), f)

In [ ]:
# Define Parameters for data extraction
START_DATE = '2019-08-01'
END_DATE = '2021-08-15'

OUTPUT_DIR = '../data/training_data/pixel_vectors'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [ ]:
# Create or extract polygons from a sampling location
sampling_file = 'lombok_v1.1.7_negative_polygons'
data_dir = '../data/sampling_locations/'
with open(os.path.join(data_dir, sampling_file + '.geojson'), 'r') as f:
    data = json.load(f)['features'] 
    
# Set rect width in pixels. Only required for point samples
num_pixels = 48
# Convert pixels to degrees. Heuristic, not geographically sound
# Better to go with slightly bigger patches that can then be cropped
rect_width = np.round((num_pixels / 100) / 111.32, 4)    

polygons = []
for feature in data:
    if feature['geometry']['type'] == 'Point':
        polygons.append(rect_from_point(feature['geometry']['coordinates'], rect_width))
    if feature['geometry']['type'] == 'MultiPolygon' or feature['geometry']['type'] == 'Polygon':
        polygons.append(feature['geometry'])

In [ ]:
# Download Sentinel Data
img_stack = []
for polygon in tqdm(polygons):
    patches = download_patch(polygon, START_DATE, END_DATE)
    for patch in patches:
        img_stack.append(patch)
print(len(img_stack), "cloud masked patches extracted")

## Make sure to set appropriate label class!
Negative sites = 0, Positive sites = 1

In [ ]:
label_class = 0
name = sampling_file

In [ ]:
subsample = 2
figure_file_path = os.path.join(OUTPUT_DIR, f"{name} - Class {label_class} - Subsample {subsample}")
plot_image_grid(img_stack[::subsample], file_path=figure_file_path)

In [ ]:
# Create pixel vectors
masked_vectors = []
for img in img_stack:
    masked_vectors.append(img[~img.mask[:,:,0]].data)
pixel_vectors = np.concatenate(masked_vectors, axis=0)

# filter black pixels. I don't know why they make it through the mask
pixel_vectors = pixel_vectors[np.mean(pixel_vectors, axis=1) > 0].tolist()
print(f"{np.shape(pixel_vectors)[0]:,} pixel vectors extracted")

# Save pixel vectors
save_pixel_vectors(data=pixel_vectors, name=name, label_class=label_class)

In [ ]:
# Plot the mean pixel spectra of the extracted dataset.
# Optional process that can take time with many samples
data = pd.DataFrame(pixel_vectors, columns=band_descriptions.keys()).melt(var_name='band', value_name='value')
plt.figure(figsize=(6,4), dpi=150, facecolor=(1,1,1))
sns.lineplot(x='band', y='value', data=data, ci="sd")
plt.title('Mean Value +/- SD')
plt.show()